## Load Data

In [19]:
from federated_gbdt.models.gbdt.private_gbdt import PrivateGBDT
from experiments.experiment_helpers.data_loader import DataLoader
from sklearn.metrics import roc_auc_score

In [2]:
dataloader = DataLoader()

# Default is 70/30 split
X_train, X_test, y_train, y_test = dataloader.load_datasets(["adult"], return_dict=False)[0]

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], dtype='int64')
Counter({0: 24720, 1: 7841})
[Data Loader] Adult Class Balance: 0.2408095574460244 



In [3]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
18081,14,3,11969,15,9,4,7,1,4,1,0,0,39,38
25159,13,3,19441,5,3,5,13,1,4,1,0,0,34,38
28376,32,3,13560,11,8,0,0,4,4,0,0,0,15,38
5516,26,5,10889,11,8,2,2,0,4,1,0,0,29,38
21754,32,4,11784,15,9,2,3,0,4,1,108,0,49,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23513,6,3,19018,8,10,4,9,1,4,0,0,0,35,38
20358,30,3,13579,6,4,2,6,0,2,1,0,0,55,-1
30292,15,3,7871,15,9,2,2,0,4,1,0,0,44,38
28443,1,3,8129,1,6,4,7,3,4,0,0,0,5,38


## XGBoost Training (No DP)

In [26]:
xgb_model = PrivateGBDT(num_trees=100, epsilon=0)
xgb_model = xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict_proba(X_test)

roc_auc_score(y_test, y_pred)

0.9227516087330172

## DP-XGBoost (FEVERLESS)

In [27]:
dp_xgb_model = PrivateGBDT(num_trees=100, epsilon=1, dp_method="gaussian_cdp")
dp_xgb_model = dp_xgb_model.fit(X_train, y_train)
y_pred = dp_xgb_model.predict_proba(X_test)

roc_auc_score(y_test, y_pred)

0.8169428179792882

## DP-TR XGBoost

In [32]:
dp_tr_model = PrivateGBDT(num_trees=100, epsilon=1, split_method="totally_random", 
                           sketch_type="uniform", dp_method="gaussian_cdp")

dp_tr_model = dp_tr_model.fit(X_train, y_train)
y_pred = dp_tr_model.predict_prob(X_test)

roc_auc_score(y_test, y_pred)

0.8832179292102441